In [2]:
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
import torch
import re
import pandas as pd
from datetime import datetime

def getRightNow():
    now = datetime.now()
    hms = now.strftime("%H:%M:%S")
    return hms
print("Imports {}".format(getRightNow()))

Imports 14:13:17


In [23]:
def getData(infile): 
    # List of Hashes
    LoH = [] 
    with open(infile, 'r') as file:
        for line in file:
            pieces = line.split("|")
            obj = {
                "letter":pieces[0], 
                "charCount":pieces[1],
                "dupCount":pieces[2],
                "brief":pieces[3],
                "productId":pieces[4],
                "activity":pieces[5],
                "desc":pieces[6]
            }
            
            LoH.append(obj)
    print("getData() got {} objs at {} from {}".format(len(LoH), getRightNow(), infile))
    return LoH
products = getData("ready_for_analysis.txt")


getData got 2509 objs at 14:50:42 from ready_for_analysis.txt


In [4]:
# List of product descriptions
product_descriptions = [
    "High-quality leather wallet for men",
    "Elegant and stylish women's handbag",
    "Durable and lightweight hiking backpack",
    "Soft and comfortable cotton t-shirt",
    "Premium stainless steel kitchen knife set"
]

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the product descriptions
encoded_inputs = tokenizer(product_descriptions, padding=True, truncation=True, return_tensors='pt')

# Get the embeddings using BERT model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Get the sentence-level embeddings (CLS token)
embeddings = outputs.last_hidden_state[:, 0, :]

# Convert embeddings to numpy array
embeddings = embeddings.numpy()

# Perform clustering using K-Means algorithm
num_clusters = 2  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(embeddings)

# Print the cluster labels for each product description
for i, desc in enumerate(product_descriptions):
    print(f"Product: {desc}\tCluster: {cluster_labels[i]}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Product: High-quality leather wallet for men	Cluster: 1
Product: Elegant and stylish women's handbag	Cluster: 0
Product: Durable and lightweight hiking backpack	Cluster: 1
Product: Soft and comfortable cotton t-shirt	Cluster: 0
Product: Premium stainless steel kitchen knife set	Cluster: 0
